In [1]:
import json
import os
import sys

import boto3
import botocore

In [2]:
bedrock = boto3.client("bedrock-runtime")

In [3]:
prompt_data = """ Tweet: "I hate it when my phone battery dies.”: Sentiment: Negative
Tweet: "My day has been great”: Sentiment: Positive
Tweet: "This is the link to the article”: Sentiment: Neutral
Tweet: "This new music video was incredible” Sentiment:
"""

In [7]:
body = json.dumps(
    {
        "inputText": prompt_data
    }
)

In [8]:
output = []
modelId = "amazon.titan-text-express-v1"  # "amazon.titan-tg1-large", "amazon.titan-text-express-v1"
accept = "application/json"
contentType = "application/json"
outputText = "\n"
try:

    response = bedrock.invoke_model_with_response_stream(
        body=body, modelId=modelId
    )
    stream = response.get("body")

    i = 1
    if stream:
        for event in stream:
            chunk = event.get("chunk")
            if chunk:
                chunk_obj = json.loads(chunk.get("bytes").decode())
                text = chunk_obj["outputText"]
                output.append(text)
                print(f"\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n")
                i += 1

except botocore.exceptions.ClientError as error:

    if error.response["Error"]["Code"] == "AccessDeniedException":
        print(
            f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n"
        )

    else:
        raise error

		**Chunk 1**
Positive

